In [ ]:
pip install geemap geopandas pydeck seaborn geetools mss earthengine-api

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 KB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.8/76.8 KB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 KB 13.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 KB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.1 

In [ ]:
# Import packages
import ee
import geemap
import geopandas as gpd
from geetools import batch
import geetools
import os
import mss
import pandas as pd
import numpy as np
import requests
import urllib.parse
import ipywidgets as widgets
import matplotlib.pyplot as plt
import seaborn
import pydeck as pdk
from datetime import datetime
from datetime import timedelta
from sklearn.metrics import r2_score
from google.colab import drive

In [ ]:
# Mount your google drive
drive.mount('/content/drive/')
# Set the directory 
os.chdir('/content/drive/My Drive/Colab Notebooks/')

Mounted at /content/drive/


In [ ]:
# Authenticate Google Earth Engine. You need to have a Google Earth Engine account for this step. 
ee.Authenticate()

# Initialize your Goggle Earth Engine account
try:
  ee.Initialize()
  print('The Earth Engine package initialized successfully!')
except ee.EEException as e:
  print('The Earth Engine package failed to initialize! Try to Authenticate EE')
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=7RaD2Bu6BTf43jjGQ04lbVCW3pJY1fiKphuO2Mf5fvM&tc=NNDGGL9R0ZtU-8_G0JZFfTuZeiK9BFrUE45GJcebt6s&cc=CPHWsao2PYrlSIRgTOAMDNIEgKlyRsFd_GqT852UdEQ

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AVHEtk5M7W1nNySPXTZIqZMkngCxHCs1DEsX-BRTtUycRUWD0bpN2lq7bLQ

Successfully saved authorization token.


In [ ]:
# Define the date range for each year
year_start = 2000
year_end = 2021

# Set a path for importing the shapefiles that contain the study area
ANWR = geemap.shp_to_ee("/content/drive/My Drive/Colab Notebooks/Files/ANWR.shp")
NPRA = geemap.shp_to_ee("/content/drive/My Drive/Colab Notebooks/Files/NPRA.shp")

# Set the coordinate reference system (Projected coordinate system for United States (USA) - Alaska)
crs = 'EPSG:3338'

# Set a region (box) around ANWR
ANWR_region = ee.Geometry.Polygon(
    [[[-140.83089376368656,66.6550212415113],
      [-140.83089376368656,70.46793183571488],
      [-149.57601095118656,70.46793183571488],
      [-149.57601095118656,66.6550212415113]]], 
    None, False
)

# Set a region (box) around NPRA
NPRA_region = ee.Geometry.Polygon(
    [[[-149.90414749199374,67.92726660651483],
      [-149.90414749199374,71.81689549403126],
      [-163.43930374199374,71.81689549403126],
      [-163.43930374199374,67.92726660651483]]],
    None, False
)
# Define a mask to clip the NDSI data by.
mask = ee.FeatureCollection("TIGER/2018/States").filter(ee.Filter.eq('NAME', 'Alaska'))

# Set the scale. The spatial resolution for the data is 500 m
scale = 500

In [ ]:
# Create an empty list that will be used to store the yearly means of the image collection 
year_ranges = []

# Create a sequence of integers from year_start to year_end + 1. The + 1 is added to include the year_end in the range (2022).
for y in range(year_start, year_end + 1):
    # Define the year starting on August, 1st and ending on May 5th of the following year - "snow year"
    start_date = ee.Date.fromYMD(y, 9, 1)
    end_date = ee.Date.fromYMD(y+1, 5, 31)
    # Create an ee.DateRange object from the start and end dates and append it to the year_ranges list. 
    # The ee.DateRange object represents a range of dates that can be used to filter Earth Engine image collections.
    date_range = ee.DateRange(start_date, end_date)
    year_ranges.append(date_range)

In [ ]:
# Fetch the MODIS collection and select the band NDSI
col = ee.ImageCollection("MODIS/006/MOD10A1").select('NDSI_Snow_Cover')

Your MODIS query returned 8327 images


In [ ]:
# Take the date range as input and returns the mean image for that year
def calculate_mean_image(date_range):
    year = ee.Date(date_range.start()).get('year')
    # Filter the image collection to include only the images within that year
    yearly_collection = col.filterDate(date_range)
    # Calculate the mean image for that year
    mean_image = yearly_collection.reduce(ee.Reducer.mean())
    # Set a property called year on each mean image to identify which year it corresponds to 
    return mean_image.set('year', year)

In [ ]:
# QA/QC: Get the date ranges for each year
for i, date_range in enumerate(year_ranges):
    start_date = date_range.start().format('YYYY-MM-dd').getInfo()
    end_date = date_range.end().format('YYYY-MM-dd').getInfo()
    print(f"Year {i+year_start}: {start_date} to {end_date}")


Year 2000: 2000-09-01 to 2001-05-31
Year 2001: 2001-09-01 to 2002-05-31
Year 2002: 2002-09-01 to 2003-05-31
Year 2003: 2003-09-01 to 2004-05-31
Year 2004: 2004-09-01 to 2005-05-31
Year 2005: 2005-09-01 to 2006-05-31
Year 2006: 2006-09-01 to 2007-05-31
Year 2007: 2007-09-01 to 2008-05-31
Year 2008: 2008-09-01 to 2009-05-31


KeyboardInterrupt: ignored

In [ ]:
# QA/QC: Print the number of iamges in the collection. Because theres 22 years in the study period, the count should be 22
snow_year_mean = ee.ImageCollection([calculate_mean_image(date_range) for date_range in year_ranges])
count_images = snow_year_mean.size()
print("Number  of images:", count_images.getInfo(), "images")

Your MODIS query returned 22 images


In [ ]:
# Clip the image collection to the study area
snowcover_NPRA = snow_year_mean.map(lambda image:image.clip(NPRA))
snowcover_ANWR = snow_year_mean.map(lambda image:image.clip(ANWR))

In [ ]:
# Export collection image to Drive
# ## Export
#tasks = batch.Export.imagecollection.toDrive(
 #           collection=snowcover_clip,
 #           folder=folder,
 #           crs=crs,
 #           scale=500,
 #           verbose=True,
 #          maxPixels=int(1e13)
  #      )

# **Create Yearly Mean Snow Cover Fraction gif**

In [ ]:
# Define arguments for animation function parameters.
video_args_NPRA = {
  'dimensions': 768,
  'region': NPRA_region,
  'framesPerSecond': 1,
  'crs': crs,
  'min': 0,
  'max': 100,
  'palette': ['0D0887', '5B02A3', '9A179B', 'CB4678', 'EB7852', 'FBB32F']
}
video_args_ANWR = {
  'dimensions': 768,
  'region': ANWR_region,
  'framesPerSecond': 1,
  'crs': crs,
  'min': 0,
  'max': 100,
  'palette': ['0D0887', '5B02A3', '9A179B', 'CB4678', 'EB7852', 'FBB32F']
}

# Get URL that will produce the animation when accessed.
snowcover_NPRA_gif =snowcover_NPRA.getVideoThumbURL(video_args_NPRA)
snowcover_ANWR_gif =snowcover_ANWR.getVideoThumbURL(video_args_ANWR)
print(snowcover_NPRA_gif)
print(snowcover_ANWR_gif)

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/videoThumbnails/50c489f4e972dfd93d36f19148b67592-c4d04199fcb2995d837f58315f34b526:getPixels


In [ ]:
# Download animation to Google Drive.
import urllib.request
NPRAgif_name = 'NPRA_yearlyMean.gif'
urllib.request.urlretrieve(snowcover_NPRA_gif, NPRAgif_name)
NPRAgif_name = 'ANWR_yearlyMean.gif'
urllib.request.urlretrieve(snowcover_ANWR_gif, ANWRgif_name)

('NPRA.gif', <http.client.HTTPMessage at 0x7fc6bca15ca0>)

Add text and scale bar to the gif

*   Change the name of the imported/ exported gif to either NPRA or ANWR
*   Change text to either the NPRA or ANWR

In [ ]:
# Set path to import gif 
in_gif = os.path.abspath('../Colab Notebooks/ANWR_yearlyMean.gif')
# Set path to export gif
out_dir = os.path.join(os.path.expanduser('/content/drive/My Drive'), 'Colab Notebooks')
# Name the exported gif
out_gif = os.path.join(out_dir, 'ANWR_yearlyMean_final.gif')
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

In [ ]:
# Add text to gif
geemap.add_text_to_gif(
    in_gif, out_gif, xy=('2%', '2%'),font_size=40, text_sequence="National Petroleum Reserve - Alaska", font_color='white',duration=950
)

geemap.add_text_to_gif(
    out_gif, out_gif, xy=('2%', '96%'),
    font_size=13, 
    text_sequence="*Average NDSI Snow Fraction", 
    font_color='white',
    duration=950
)

# Add dates
geemap.add_text_to_gif(
    in_gif,
    out_gif,
    xy=('2%', '10%'),
    text_sequence=2000,
    font_size=40,
    font_color='white',
    duration=950
)
geemap.show_image(out_gif)

# Create color bar 
from geemap import *
width = 300
height = 30
palette = ['0D0887', '5B02A3', '9A179B', 'CB4678', 'EB7852', 'FBB32F']
labels = [0, 100]
colorbar = create_colorbar(
    width=width,
    height=height,
    palette=palette,
    vertical=True,
    add_labels=True,
    add_ticks=False,
    font_size=30,
    labels=labels,
    font_color='white'
)

show_image(colorbar)

# Add color bar to gif 
add_image_to_gif(
    out_gif, out_gif, in_image=colorbar, xy=('5%', '16%'), image_size=(250, 250)
)

geemap.show_image(out_gif)